In [1]:
import asf_search as asf

results = asf.search(
    platform=[asf.PLATFORM.SENTINEL1],
    processingLevel=[asf.PRODUCT_TYPE.GRD_HD],
    start="2022-01-01",
    end="2022-01-02",
    intersectsWith="POLYGON((-120 35, -120 36, -119 36, -119 35, -120 35))"
)

print("Found scenes:", len(results))
results[0].download(path="./data")


["'type': 'REVERSE': 'report': Reversed polygon winding order"]


Found scenes: 3


/opt/miniconda3/envs/photo/lib/python3.10/site-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ./data/S1A_IW_GRDH_1SDV_20220101T135219_20220101T135244_041268_04E7A6_6B2B.zip
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')


In [2]:
import os
from pyroSAR import identify
from spatialist import Raster
import numpy as np
import matplotlib.pyplot as plt

data_dir = "./data"
output_dir = "./eda_results"
os.makedirs(output_dir, exist_ok=True)

# loop over all downloaded zip files
for f in os.listdir(data_dir):
    if f.endswith(".zip"):
        path = os.path.join(data_dir, f)

        # identify metadata
        meta = identify(path)
        print(f"Scene: {f}")
        print(meta)

        # read VV band
        vv_files = meta.findfiles('^measurement.*vv.*tiff$')
        if not vv_files:
            continue
        vv_path = vv_files[0]

        with Raster(vv_path) as ras:
            arr = ras.array()
            arr = arr.astype(np.float32)

            # statistics
            print("Shape:", arr.shape)
            print("Min:", np.min(arr), "Max:", np.max(arr))
            print("Mean:", np.mean(arr), "Std:", np.std(arr))

            # save histogram
            plt.figure()
            plt.hist(arr.flatten(), bins=200, range=(np.min(arr), np.max(arr)))
            plt.title(f"Histogram VV - {f}")
            plt.xlabel("Backscatter Amplitude")
            plt.ylabel("Frequency")
            plt.savefig(os.path.join(output_dir, f"{f}_histogram.png"))
            plt.close()

            # save quick image preview
            plt.figure(figsize=(10,10))
            plt.imshow(arr, cmap='gray')
            plt.title(f"VV Preview - {f}")
            plt.colorbar(label="Amplitude")
            plt.savefig(os.path.join(output_dir, f"{f}_preview.png"))
            plt.close()


Scene: S1A_IW_GRDH_1SDV_20220101T135219_20220101T135244_041268_04E7A6_6B2B.zip
pyroSAR ID object of type SAFE
acquisition_mode: IW
coordinates: [(-116.354172, 34.18055), (-119.091644, 34.583302), (-118.768211, 36.083237), (-115.979294, 35.681915)]
cycleNumber: 250
frameNumber: 321446
lines: 16716
orbit: D
orbitNumber_abs: 41268
orbitNumber_rel: 71
polarizations: ['VV', 'VH']
product: GRD
projection: +proj=longlat +datum=WGS84 +no_defs
samples: 25576
sensor: S1A
spacing: (10.0, 10.0)
start: 20220101T135219
stop: 20220101T135244


In [ ]:
import os
import zipfile
from pyroSAR import identify
from spatialist import Raster
import numpy as np
import matplotlib.pyplot as plt

data_dir = "./data"
output_dir = "./eda_results"
os.makedirs(output_dir, exist_ok=True)

for f in os.listdir(data_dir):
    if not f.endswith(".zip"):
        continue

    zip_path = os.path.join(data_dir, f)
    meta = identify(zip_path)
    print(f"Scene: {f}")
    print(meta)

    # extract zip fully
    extract_dir = os.path.join(data_dir, f"{f}_extracted")
    if not os.path.exists(extract_dir):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

    # recursively find VV TIFF
    vv_files = []
    for root, dirs, files in os.walk(extract_dir):
        for file in files:
            if file.lower().endswith(".tiff") and "vv" in file.lower():
                vv_files.append(os.path.join(root, file))

    if not vv_files:
        print("No VV band found, skipping.")
        continue

    # pick first VV file
    vv_path = os.path.abspath(vv_files[0])
    print("Using VV path:", vv_path)

    # sanity check: file must exist
    if not os.path.exists(vv_path):
        print("VV file not found on disk:", vv_path)
        continue

    # read raster without using /vsizip/
    with Raster('/Users/yifengwu/Documents/fun_project/sar/data/S1A_IW_GRDH_1SDV_20220101T135219_20220101T135244_041268_04E7A6_6B2B.zip/S1A_IW_GRDH_1SDV_20220101T135219_20220101T135244_041268_04E7A6_6B2B.SAFE/measurement/s1a-iw-grd-vv-20220101t135219-20220101t135244-041268-04e7a6-001.tiff', list_separate=False) as ras:
        vv_arr = ras.array().astype(np.float32)

    print("Shape:", vv_arr.shape)
    print("Min:", np.min(vv_arr), "Max:", np.max(vv_arr))
    print("Mean:", np.mean(vv_arr), "Std:", np.std(vv_arr))

    # save histogram
    plt.figure()
    plt.hist(vv_arr.flatten(), bins=200)
    plt.title(f"Histogram VV - {f}")
    plt.xlabel("Backscatter Amplitude")
    plt.ylabel("Frequency")
    plt.savefig(os.path.join(output_dir, f"{f}_VV_histogram.png"))
    plt.close()

    # save preview
    plt.figure(figsize=(10,10))
    plt.imshow(vv_arr, cmap='gray')
    plt.title(f"VV Preview - {f}")
    plt.colorbar(label="Amplitude")
    plt.savefig(os.path.join(output_dir, f"{f}_VV_preview.png"))
    plt.close()


Scene: S1A_IW_GRDH_1SDV_20220101T135219_20220101T135244_041268_04E7A6_6B2B.zip
pyroSAR ID object of type SAFE
acquisition_mode: IW
coordinates: [(-116.354172, 34.18055), (-119.091644, 34.583302), (-118.768211, 36.083237), (-115.979294, 35.681915)]
cycleNumber: 250
frameNumber: 321446
lines: 16716
orbit: D
orbitNumber_abs: 41268
orbitNumber_rel: 71
polarizations: ['VV', 'VH']
product: GRD
projection: +proj=longlat +datum=WGS84 +no_defs
samples: 25576
sensor: S1A
spacing: (10.0, 10.0)
start: 20220101T135219
stop: 20220101T135244
Using VV path: /Users/yifengwu/Documents/fun_project/sar/data/S1A_IW_GRDH_1SDV_20220101T135219_20220101T135244_041268_04E7A6_6B2B.zip_extracted/S1A_IW_GRDH_1SDV_20220101T135219_20220101T135244_041268_04E7A6_6B2B.SAFE/measurement/s1a-iw-grd-vv-20220101t135219-20220101t135244-041268-04e7a6-001.tiff
Shape: (16716, 25576)
Min: 0.0 Max: 16117.0
Mean: 138.00041 Std: 83.43339


KeyboardInterrupt: 